In [91]:
import math

In [92]:
def divides(n, d):
    return not n % d

In [93]:
def swap(n, x, y):
    n[x], n[y] = n[y], n[x]


def partition(n, low, high):
    pivot, id_x = n[high], low
    
    for i in range(low, high):
        if n[i] <= pivot:
            swap(n, i, id_x)
            id_x += 1
    
    swap(n, id_x, high)
    return id_x

def quick_sort_recurr(n, low, high):
    if len(n) == 1:
        return n
    if low < high:
        center = partition(n, low, high)
        quick_sort_recurr(n, low, center - 1), quick_sort_recurr(n, center + 1, high)
    return n

def quick_sort(n):
    return quick_sort_recurr(n, 0, len(n) - 1)


In [94]:
def sieve(max_num: int):    
    primes = [True for x in range(2, max_num)]
    
    for i in range(4, max_num, 2):
        primes[i - 2] = False
    
    for i in range(3, max_num):
        if not primes[i - 2]:
            continue
        for j in range(i * i, max_num, i):
            primes[j - 2] = False
    return primes 

def factorate(num: int, cache: list = None):
    if not cache:
        cache = sieve(int(num / 2) + 1)
        
    factors = list()
    
    while num != 1:
        for prime, i in enumerate(cache):
            if num % (prime + 2):
                continue
            while not num % (prime + 2):
                factors.append(prime + 2)
                num = int(num / (prime + 2))
        if num != 1:
            factors.append(num)
        break
    
    return factors

In [95]:
def merge(first_list, second_list):    
    result = list()
    
    while first_list or second_list:
        if not first_list:
            result.append(second_list.pop(0))
        elif not second_list:
            result.append(first_list.pop(0))
        elif first_list[0] < second_list[0]:
            result.append(first_list.pop(0))
        else:
            result.append(second_list.pop(0))
    return result

In [103]:
def factor_product(arr: list):
    size = len(arr)
    while size > 1:
        arr[size - 2] = merge(arr[size - 2], arr.pop())
        size -= 1
        
    return arr[0]

In [104]:
def read_fraction():
    n, d = ([int(x) for x in input().split()] for _ in range(2))
    return frac(n, d)

In [105]:
def arrange(n, p):
    return [x for x in range(n, n-p, -1)]

In [99]:
def binary_search(n: list, val: int):
    low, center, high = 0, 0, len(n) - 1

    while low <= high:
        center = (high + low) // 2

        if n[center] < val:
            low = center + 1
        elif n[center] > val:
            high = center - 1
        else:
            return center

    return center

In [100]:
def insort(n: list, val: int):
    new = n.copy()
    new.insert(binary_search(n, val), val)
    return new

In [101]:
class frac:
    prime_cache = sieve(10000)

    def __init__(self, n: int = None, d: int = None, composite: tuple = None):
        if n and d is not None:
            composite = (factorate(n, self.prime_cache), factorate(d, self.prime_cache))
            self.n, self.d = self._simplify(composite)
        else:
            self.n, self.d = self._simplify(composite)

    @staticmethod
    def _simplify(composite: tuple):
        n, d = composite
        id_x, id_y = 0, 0
        size_x, size_y = len(n), len(d)

        result_n, result_d = (list(), list())

        while id_x < size_x or id_y < size_y:
            if id_x >= size_x:
                result_d.append(d[id_y])
                id_y += 1
            elif id_y >= size_y:
                result_n.append(n[id_x])
                id_x += 1
            elif n[id_x] < d[id_y]:
                result_n.append(n[id_x])
                id_x += 1
            elif n[id_x] > d[id_y]:
                result_d.append(d[id_y])
                id_y += 1
            else:
                id_x += 1
                id_y += 1

        return result_n, result_d

    def __mul__(self, other):
        if isinstance(other, frac):
            n = merge(self.n, other.n)
            d = merge(self.d, other.d)
        elif isinstance(other, int):
            n = insort(self.n, other)
            d = self.d
        else:
            n = self.n
            d = self.d

        return frac(composite=(n, d))

    def __add__(self, other):
        if isinstance(other, frac):
            n = math.prod(self.n) * math.prod(other.d) + math.prod(self.d) * math.prod(other.n)
            n = factorate(n, self.prime_cache)
            d = merge(self.d, other.d)
        elif isinstance(other, int):
            n = math.prod(self.n) + math.prod(self.d) * other
            n = factorate(n, self.prime_cache)
            d = self.d
        else:
            n = self.n
            d = self.d

        return frac(composite=(n, d))

    def __sub__(self, other):
        if isinstance(other, frac):
            n = math.prod(self.n) * math.prod(other.d) - math.prod(self.d) * math.prod(other.n)
            n = factorate(n, self.prime_cache)
            d = merge(self.d, other.d)
        elif isinstance(other, int):
            n = math.prod(self.n) - math.prod(self.d) * other
            n = factorate(n, self.prime_cache)
            d = self.d
        else:
            n = self.n
            d = self.d

        return frac(composite=(n, d))

    def __rsub__(self, other):
        if isinstance(other, frac):
            n = -math.prod(self.n) * math.prod(other.d) + math.prod(self.d) * math.prod(other.n)
            n = factorate(n, self.prime_cache)
            d = merge(self.d, other.d)
        elif isinstance(other, int):
            n = -math.prod(self.n) + math.prod(self.d) * other
            n = factorate(n, self.prime_cache)
            d = self.d
        else:
            n = self.n
            d = self.d

        return frac(composite=(n, d))

    __rmul__ = __mul__
    __radd__ = __add__

    def __str__(self):
        if not self.d:
            return str(math.prod(self.n))
        else:
            return f"{math.prod(self.n)}/{math.prod(self.d)}"

    @property
    def decimal(self):
        return math.prod(self.n)/math.prod(self.d)